#### Method 1 as advised by the Uadcity guide, clicking 'Insert to code'-> 'Insert pandas DataFrame'

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6244885cda944e7ebb1285a93c7c42bd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='KE2C-XN165hErEFnJNrUpzaVMTCI67o6SOqm2jwV2gZY',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_6244885cda944e7ebb1285a93c7c42bd.get_object(Bucket='sparkify-donotdelete-pr-10odilmza0szjs',Key='medium-sparkify-event-data.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html




In [2]:
df_data_2 = pd.read_json(body, orient='values')
# df_data_2.head()

ValueError: Trailing data

#### Method 1.1 using lines= True

In [3]:
df_data_3 = pd.read_json(body, orient='values',lines=True )
df_data_3.head()

""


#### Method 2: using ibmos2spark

In [4]:
!pip install ibmos2spark

In [5]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 205kB/s  eta 0:00:01
     |████████████████████████████████| 204kB 48.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [6]:
import ibmos2spark
from pyspark import SparkContext, SparkConf

In [7]:
conf = SparkConf().setAppName('Sparkify').setMaster('local')
sc = SparkContext(conf=conf)

In [8]:
import ibmos2spark

# @hidden_cell
# credentials = {
#     'IAM_SERVICE_ID': 'iam-ServiceId-e7c76399-a89e-4974-9557-ba5c60d5daa1',
#     'ibm_api_key': 'KE2C-XN165hErEFnJNrUpzaVMTCI67o6SOqm2jwV2gZY',
#     'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
#     'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
#     'BUCKET': 'sparkify-donotdelete-pr-10odilmza0szjs',
#     'FILE': 'medium-sparkify-event-data.json'}

credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'api_key': 'KE2C-XN165hErEFnJNrUpzaVMTCI67o6SOqm2jwV2gZY',
    'service_id': 'iam-ServiceId-e7c76399-a89e-4974-9557-ba5c60d5daa1'
}

configuration_name = 'os_a68347975ac949879ccd31e5ca85693c_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



In [10]:
df_data_1 = spark.read.json(cos.url('medium-sparkify-event-data.json','sparkify-donotdelete-pr-10odilmza0szjs' ))
df_data_1.take(5)

Py4JJavaError: An error occurred while calling o26.json.
: java.io.IOException: No FileSystem for scheme: cos
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:547)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:545)
	at scala.collection.TraversableLike$$anonfun$f

In [11]:
df_data_1 = spark.read.json(cos.url('medium-sparkify-event-data.json', 'Sparkify'))
df_data_1.take(5)

Py4JJavaError: An error occurred while calling o61.json.
: java.io.IOException: No FileSystem for scheme: cos
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:547)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:545)
	at scala.collection.TraversableLike$$anonfun$f